In [11]:
from bqskit.ext import qiskit_to_bqskit as converter_to_bqskit
from pennylane import from_qiskit as converter_to_pennylane
from pytket.extensions.qiskit import qiskit_to_tk as converter_to_pytket
from itertools import combinations
from qiskit import QuantumCircuit
from copy import deepcopy
from typing import List, Callable, Dict, Any
from typing import List, Dict, Any, Callable, Tuple, Optional
from pathlib import Path
import inspect
import time
import sys
import uuid
import json
import traceback
import os
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import Session, Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Section: Circuit
qr = QuantumRegister(3, name='qr')
cr = ClassicalRegister(3, name='cr')
qc = QuantumCircuit(qr, cr, name='qc')
# TIMESTAMP: 1737648255.013264

# Apply gate operations
# <START_GATES>

qc.mcrz(4.172311, [qr[0], qr[1]], qr[2])

qc.measure(qr, cr)
print(qc.draw())


def optimize_with_pytket(
        tkqc, var_name: str, output_dir: Optional[str] = None):
    """Optimize a pytket circuit and export as qasm."""
    from pytket.qasm import circuit_to_qasm_str
    from pytket.passes import FullPeepholeOptimise, PeepholeOptimise2Q, RemoveRedundancies, EulerAngleReduction, KAKDecomposition, CliffordPushThroughMeasures, FlattenRegisters, PauliSimp, GreedyPauliSimp, OptimisePhaseGadgets, ZXGraphlikeOptimisation
    from pytket.circuit import OpType
    optimization_passes = {
        'FullPeepholeOptimise': FullPeepholeOptimise(),
        'PeepholeOptimise2Q': PeepholeOptimise2Q(),
        'RemoveRedundancies': RemoveRedundancies(),
        'EulerAngleReduction':
        EulerAngleReduction(q=OpType.Rz, p=OpType.Rx),
        'KAKDecomposition': KAKDecomposition(),
        'CliffordPushThroughMeasures': CliffordPushThroughMeasures(),
        'FlattenRegisters': FlattenRegisters(),
        'PauliSimp': PauliSimp(),
        'GreedyPauliSimp': GreedyPauliSimp(),
        'OptimisePhaseGadgets': OptimisePhaseGadgets(),
        'ZXGraphlikeOptimisation': ZXGraphlikeOptimisation()}
    for opt_pass_name, optimization_pass in optimization_passes.items():
        i_qc = deepcopy(tkqc)
        print(f'Applying {opt_pass_name}...')
        optimization_pass.apply(i_qc)
        i_opt_circuit_qasm = circuit_to_qasm_str(i_qc, header='qelib1',
                                                 maxwidth=200)
        if output_dir is not None:
            file_path_pytket = Path(
                output_dir) / f'{var_name}_{opt_pass_name}_pytket.qasm'
        else:
            current_file = Path(__file__)
            file_stem = current_file.stem
            file_path_pytket = current_file.with_name(
                f'{file_stem}_{var_name}_{opt_pass_name}_pytket.qasm')
        with open(file_path_pytket, 'w') as f:
            f.write(i_opt_circuit_qasm)
        print(
            f'Saved Optimized Pytket circuit ({opt_pass_name}) to {file_path_pytket}'
        )


def get_functions(prefix: str) -> Dict[str, Callable]:
    """
    Finds all functions in the current module that start with the given prefix
    and returns a list of these functions without executing them.

    Args:
        prefix (str): The prefix to filter functions by.

    Returns:
        Dict[str, Callable]: A dictionary with the platform name as key and
        the function as value.
    """
    current_module = sys.modules[__name__]
    functions = inspect.getmembers(current_module, inspect.isfunction)
    filtered_functions = [(name, func) for name, func in functions if name.
                          startswith(prefix)]
    dict_filtered_functions = {name.split('_')[-1]: func for name, func in
                               filtered_functions}
    return dict_filtered_functions


def get_copy_of_all_circuits_vars() -> Dict[str, QuantumCircuit]:
    """
    Returns a copy of all QuantumCircuit variables in the current module.

    Returns:
        Dict[str, QuantumCircuit]: A dictionary with variable names as keys and QuantumCircuit objects as values.
    """
    all_circuits_vars = {
        var_name: deepcopy(var_value) for var_name, var_value
        in globals().items() if isinstance(var_value, QuantumCircuit)}
    return all_circuits_vars


def oracle_optimizer(output_dir: str = None) -> None:
    """Run all the oracle functions to optimize the QC and export QASM files.

    If no output directory is provided, the QASM files will be saved in the
    current directory.
    """
    optimize_calls = get_functions(prefix='optimize_with_')
    all_circuits_vars = get_copy_of_all_circuits_vars()

    def converter_to_qiskit(qc: QuantumCircuit) -> QuantumCircuit:
        return qc
    converters_calls = {'bqskit': converter_to_bqskit,
                        'pytket': converter_to_pytket,
                        'pennylane': converter_to_pennylane,
                        'qiskit': converter_to_qiskit}
    if output_dir:
        abs_path_output_dir = Path(output_dir).resolve()
    else:
        abs_path_output_dir = None
    for var_name, target_qc in all_circuits_vars.items():
        for platform, platform_specific_optimize_call in optimize_calls.items(
        ):
            converter_call = converters_calls.get(platform)
            qc_to_convert = deepcopy(target_qc)
            platform_specific_qc = converter_call(qc_to_convert)
            qasm_file = platform_specific_optimize_call(
                platform_specific_qc, var_name=var_name,
                output_dir=abs_path_output_dir)


oracle_optimizer()

                                                      ┌─┐                      
qr_0: ──■───────────────────────────────■─────────────┤M├──────────────────────
        │                               │             └╥┘                ┌─┐   
qr_1: ──┼───────────────■───────────────┼──────────────╫───■─────────────┤M├───
      ┌─┴─┐┌─────────┐┌─┴─┐┌─────────┐┌─┴─┐┌─────────┐ ║ ┌─┴─┐┌─────────┐└╥┘┌─┐
qr_2: ┤ X ├┤ Unitary ├┤ X ├┤ Unitary ├┤ X ├┤ Unitary ├─╫─┤ X ├┤ Unitary ├─╫─┤M├
      └───┘└─────────┘└───┘└─────────┘└───┘└─────────┘ ║ └───┘└─────────┘ ║ └╥┘
cr: 3/═════════════════════════════════════════════════╩══════════════════╩══╩═
                                                       0                  1  2 
Applying FullPeepholeOptimise...


RuntimeError: Operation is not a gate: Unitary1qBox

In [32]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

from pytket.extensions.qiskit import qiskit_to_tk
from pytket.passes import FullPeepholeOptimise, PeepholeOptimise2Q
from pytket.qasm import circuit_to_qasm_str

qr = QuantumRegister(3, name='qr')
cr = ClassicalRegister(3, name='cr')
qc = QuantumCircuit(qr, cr, name='qc')

qc.mcrz(1.23, [qr[0], qr[1]], qr[2])
# qc.measure(qr, cr)
print(qc.draw())

# Convert Qiskit circuit to Pytket circuit
tkqc = qiskit_to_tk(qc)

# Apply Full Peephole Optimization
FullPeepholeOptimise().apply(tkqc)
# CRASH: RuntimeError: Operation is not a gate: Unitary1qBox

# Apply Peephole Optimization 2Q
# PeepholeOptimise2Q().apply(tkqc)
# Working: No crash

# Export optimized circuit as QASM
opt_circuit_qasm = circuit_to_qasm_str(tkqc, header='qelib1', maxwidth=200)
print(opt_circuit_qasm)

                                                                      
qr_0: ──■───────────────────────────────■─────────────────────────────
        │                               │                             
qr_1: ──┼───────────────■───────────────┼───────────────■─────────────
      ┌─┴─┐┌─────────┐┌─┴─┐┌─────────┐┌─┴─┐┌─────────┐┌─┴─┐┌─────────┐
qr_2: ┤ X ├┤ Unitary ├┤ X ├┤ Unitary ├┤ X ├┤ Unitary ├┤ X ├┤ Unitary ├
      └───┘└─────────┘└───┘└─────────┘└───┘└─────────┘└───┘└─────────┘
cr: 3/════════════════════════════════════════════════════════════════
                                                                      


RuntimeError: Operation is not a gate: Unitary1qBox

In [27]:
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(tkqc)